### YelpDataCollection
Queries the Yelp search API to get those businesses that 'gluten_free' category for a given zip code.  
https://www.yelp.com/developers/documentation/v3/business_search

In [1]:
#-- Import Libraries
import pandas as pd
import os
import requests

# Yelp API key in secrets.py; .gitignore prevents the secrets.py from being pushed to GitHub
from secrets import yelpKey


In [2]:
def getDataForZipcode(searchZipCode):
    ''' Searches the Yelp API to get the business that satisfy the 'gluten_free' search term
    
    Accepts : searchZipCode (str) zip code to search for records within
    
    Returns : (dictionary) contains information of business for the zip code
                ID: Unique Yelp ID for the business
                Name: Name of the business
                ZipCode: Location of the business
                Latitude: coordinate of the business location
                Longitude: coordinate of the business location
                Price: Price level of the business. Value is one of $, $$, $$$, $$$$ and NA
                Rating: Rating for this business (value ranges from 1, 1.5, ... 4.5, 5)
    '''
    
    #- Prepare Search
    # Source Url
    baseYelpUrl = "https://api.yelp.com/v3/businesses/search"

    # API Key passed through header
    headers = {
            'Authorization': 'Bearer %s' % yelpKey,
    }
    
    # Search Term
    searchTerm = 'gluten_free'
    
    # Dictionary stores data
    yelpData = {
        'ID': [],
        'Name': [],
        'Zipcode': [],
        'Latitude': [],
        'Longitude': [],
        'Price' : [],
        'Rating' : []
    }
    
    
    #- Display Message
    print(" ")
    print(f"Started search of Yelp API zip code: {searchZipCode}...")
    
    
    #- Search
    #  API limits 50 records being returned at once; must loop and request offset of results to get all records
    recordLimit = 50
    currentOffset = 0
    hasMoreData = True
    
    while hasMoreData == True:
        
        #- Prepare Parameters
        parameters = {
            'location': searchZipCode,
            'term': 'gluten_free',
            'limit': recordLimit,
            'radius': 3000,
            'offset': currentOffset,
            }
        
        
        #- Request
        print(f"  Requesting data. Offset: {currentOffset}")
        
        response = requests.request('GET', baseYelpUrl, headers=headers, params=parameters)
        
        
        #- Check Response
        if (response.status_code == requests.codes.ok):
            
            # Get Json from Response
            responseJson = response.json()
            
            
            # Search Businesses
            for business in responseJson['businesses']:
                
                # Determine Use Business
                useBusiness = checkBusinessForUsage(business, searchZipCode, searchTerm)
                
                if (useBusiness == True):
                    
                    # Populate Dictionary with Business Information
                    yelpData['ID'].append(business['id'])
                    yelpData['Name'].append(business['name'])
                    yelpData['Zipcode'].append(business['location']['zip_code'])
                    
                    yelpData['Latitude'].append(business['coordinates']['latitude'])
                    yelpData['Longitude'].append(business['coordinates']['longitude'])
                    
                    yelpData['Price'].append(getPriceForBusiness(business))
                    yelpData['Rating'].append(business['rating'])
          
        
        #- Prepare for Next search
        # API only supports 50 records at a time; must query with offset
        currentOffset = (currentOffset + recordLimit)
        
        if (currentOffset > responseJson['total']):
            print(f"Collected all data. Current Offset: {currentOffset}  Total: {responseJson['total']}")
            hasMoreData = False
    
                  
    #- Metadata on Data
    print(" ")
    print(f"Search Zipcode: {searchZipCode}")
    print(f"Total businesses: {len(yelpData['ID'])}")
               
          
    #- Return data from function
    return yelpData


In [3]:
def checkBusinessForUsage(businessInfo, searchZipcode, searchTerm):
    ''' Determines if the business can be used in the Analysis
    
    Accepts : businessInfo (dictionary) contains the metadata for individual business 
                searchZipCode (str) zip code searching for data within
                searchTerm (str) category used with the search; ensure found
    
    Return : bool TRUE- business meets critera, able to use FALSE- unable to use business
    '''
    
    #- Check Within Search Zipcode
    businessZipCode = businessInfo['location']['zip_code']
    
    if (businessZipCode != searchZipcode):
        return False
    
    
    
    # NOTE: Found that when checking for the category there were very few records that satisfy this
    
    #- Check Search Term
#     foundSearchTerm = False
    
#     for category in businessInfo['categories']:
        
#         if (category['alias'] == searchTerm):
#             foundSearchTerm = True
#             break
    
    
    return True
     

In [4]:
def getPriceForBusiness(businessInfo):
    ''' Gets the price for a business; not all businesses contain this property within the JSON;
    when not found just uses NA.
    
    Accepts : businessInfo (dictionary) metadata on an individual business
    
    Returns : (num) value from price tag
    '''
    try:
        
        return businessInfo['price']
    
    except:
        return 'NA'

In [5]:
#-- Collect Data, Multiple Datasets

#- UserName
userName = "Riley"

#- Get File of Random Zipcodes
print("--->")
print("Getting random zipcodes...")


fileName = f'RandomZipCodes_{userName}.csv'
randomZipcodesPath = os.path.join(".", "Output", fileName)

randomZipcodes_df = pd.read_csv(randomZipcodesPath)


#- Collect Data
for index, row in randomZipcodes_df.iterrows():
    
    #- Get Zipcode
    searchZipCode = str(row[1])

    
    #- Get Data from Yelp
    yelpDataForZipCode = getDataForZipcode(searchZipCode)


    #- Create DataFrame
    yelpData_df = pd.DataFrame(yelpDataForZipCode)

    
    #-- Export Data
    dataExportPath = os.path.join(".", "Output", f"YelpData_{searchZipCode}.csv")

    yelpData_df.to_csv(dataExportPath)

    print(f"Exported data to disk. Path: {dataExportPath}")

    
#- Complete Message
print(" ")
print("Completed getting information for zipcodes")


--->
Getting random zipcodes...
 
Started search of Yelp API zip code: 3752...
  Requesting data. Offset: 0
Collected all data. Current Offset: 50  Total: 0
 
Search Zipcode: 3752
Total businesses: 0
Exported data to disk. Path: ./Output/YelpData_3752.csv
 
Started search of Yelp API zip code: 12204...
  Requesting data. Offset: 0
Collected all data. Current Offset: 50  Total: 13
 
Search Zipcode: 12204
Total businesses: 7
Exported data to disk. Path: ./Output/YelpData_12204.csv
 
Started search of Yelp API zip code: 47139...
  Requesting data. Offset: 0
Collected all data. Current Offset: 50  Total: 0
 
Search Zipcode: 47139
Total businesses: 0
Exported data to disk. Path: ./Output/YelpData_47139.csv
 
Started search of Yelp API zip code: 77868...
  Requesting data. Offset: 0
Collected all data. Current Offset: 50  Total: 1
 
Search Zipcode: 77868
Total businesses: 1
Exported data to disk. Path: ./Output/YelpData_77868.csv
 
Started search of Yelp API zip code: 55951...
  Requesting d

  Requesting data. Offset: 50
Collected all data. Current Offset: 100  Total: 91
 
Search Zipcode: 21205
Total businesses: 2
Exported data to disk. Path: ./Output/YelpData_21205.csv
 
Started search of Yelp API zip code: 44143...
  Requesting data. Offset: 0
Collected all data. Current Offset: 50  Total: 9
 
Search Zipcode: 44143
Total businesses: 5
Exported data to disk. Path: ./Output/YelpData_44143.csv
 
Started search of Yelp API zip code: 1270...
  Requesting data. Offset: 0
Collected all data. Current Offset: 50  Total: 3
 
Search Zipcode: 1270
Total businesses: 0
Exported data to disk. Path: ./Output/YelpData_1270.csv
 
Started search of Yelp API zip code: 90301...
  Requesting data. Offset: 0
  Requesting data. Offset: 50
  Requesting data. Offset: 100
  Requesting data. Offset: 150
  Requesting data. Offset: 200
Collected all data. Current Offset: 250  Total: 203
 
Search Zipcode: 90301
Total businesses: 9
Exported data to disk. Path: ./Output/YelpData_90301.csv
 
Started sear

In [6]:
#-- Collect Data

# Zip Code to search
# Able to pass this as function in future
searchZipCode = "92104"


#- Get Data From Yelp
yelpDataForZipCode = getDataForZipcode(searchZipCode)


#- Create DataFrame
yelpData_df = pd.DataFrame(yelpDataForZipCode)

 
Started search of Yelp API zip code: 92104...
  Requesting data. Offset: 0
  Requesting data. Offset: 50
  Requesting data. Offset: 100
  Requesting data. Offset: 150
  Requesting data. Offset: 200
  Requesting data. Offset: 250
Collected all data. Current Offset: 300  Total: 272
 
Search Zipcode: 92104
Total businesses: 120


In [8]:
#-- Export Data
dataExportPath = os.path.join(".", "Output", f"YelpData_{searchZipCode}.csv")

yelpData_df.to_csv(dataExportPath)

print(f"Exported data to disk. Path: {dataExportPath}")

Exported data to disk. Path: ./Output/YelpData_92104.csv


In [9]:
#-- Preview Data
yelpData_df.head(200)

,ID,Name,Zipcode,Latitude,Longitude,Price,Rating
0,0JmO9IUPf4qHa-lqTe8c3Q,Curryosity,92104,32.729480,-117.129270,NA,4.0
1,FRfUgOKobN0foq12s9OUTA,Mike Hess Brewing Company,92104,32.747702,-117.128543,$,4.5
2,jLND0q_AFpsxY8CQ3njKeQ,Bread and Cheese Catering,92104,32.750252,-117.136963,NA,5.0
3,nap_QrXJTEzJgj0_VbxBBg,619 Spirits North Park,92104,32.750151,-117.129790,$$,4.5
4,-BZ9L7G1OOUsb_uMYYuYeg,Butter Up Bakery,92104,32.750126,-117.137421,$$,5.0
5,KX1aYCfF_bKSSu5lCrgQwA,Couch Potatoes,92104,32.730572,-117.113571,NA,5.0
6,RXn_bE7f3eSpCvDOY06tQQ,One Door North,92104,32.741550,-117.130360,$$,4.0
7,YLV7ZZoyOAXW3eqidGXBZA,The Smoking Goat,92104,32.741353,-117.130230,$$,4.0
8,2viC5B-psszDSGKHkp8I2Q,Dunedin New Zealand Eats,92104,32.742340,-117.129860,$$,4.0
9,H3XluJQlArmynJc2YBErzg,Thorn St Brewery,92104,32.739350,-117.125484,$,4.5
